# Import the dataset and explore the data

In [2]:
#maybe 0.0 it's a nan, some values we don't have, but exists

#Claim Identifier: should be the fistr column
#Claim Injury Type: should be the last column
#First Hearing Date: blank means no hearing yet, can we input? I think not
#IME-4 Count: Few info, just for who has an attorney? maybe we should delete for modeling purposes
#OIICS Nature of Injury Description: no data
#Agreement Reached: Can't input
#WCB Decision: Can't input

#Age at Injury: if it's a workplace it means ages like 5 or 9 don't make sense, we should delete them
#Average Weekly Wage: make sense they don't receive money?
#Birth Year: with a lot of 0.0, we don't need birth year, we have age
#Carrier Type: we have a lot of UNKNOW, so maybe we can conclude 0.0 or nothing mean missing values
#Claim Identifier: 292668076 is repeated
#WCIO Part Of Body Code: -9 is an error?

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# data partition
from sklearn.model_selection import train_test_split

#filter methods
# spearman 
# chi-square
import scipy.stats as stats
from scipy.stats import chi2_contingency

#wrapper methods
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import RFE

# embedded methods
from sklearn.linear_model import LassoCV

from sklearn.metrics import confusion_matrix

In [4]:
train_data = pd.read_csv('train_data.csv', sep = ',')
test_data = pd.read_csv('test_data.csv', sep = ',')

C:\Users\lopes\AppData\Local\Temp\ipykernel_1084\1422952055.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv('train_data.csv', sep = ',')


In [5]:
train_data.head()

,Accident Date,Age at Injury,Alternative Dispute Resolution,Assembly Date,Attorney/Representative,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,...,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Agreement Reached,WCB Decision,Number of Dependents
0,2019-12-30,31.0,N,2020-01-01,N,0.00,1988.0,2019-12-31,NaN,NEW HAMPSHIRE INSURANCE CO,...,27.0,FROM LIQUID OR GREASE SPILLS,10.0,CONTUSION,62.0,BUTTOCKS,13662,0.0,Not Work Related,1.0
1,2019-08-30,46.0,N,2020-01-01,Y,1745.93,1973.0,2020-01-01,2020-01-14,ZURICH AMERICAN INSURANCE CO,...,97.0,REPETITIVE MOTION,49.0,SPRAIN OR TEAR,38.0,SHOULDER(S),14569,1.0,Not Work Related,4.0
2,2019-12-06,40.0,N,2020-01-01,N,1434.80,1979.0,2020-01-01,NaN,INDEMNITY INSURANCE CO OF,...,79.0,OBJECT BEING LIFTED OR HANDLED,7.0,CONCUSSION,10.0,MULTIPLE HEAD INJURY,12589,0.0,Not Work Related,6.0
3,NaN,NaN,NaN,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-12-30,61.0,N,2020-01-01,N,NaN,1958.0,2019-12-31,NaN,STATE INSURANCE FUND,...,16.0,"HAND TOOL, UTENSIL; NOT POWERED",43.0,PUNCTURE,36.0,FINGER(S),12603,0.0,Not Work Related,1.0


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593471 entries, 0 to 593470
Data columns (total 33 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Accident Date                       570337 non-null  object 
 1   Age at Injury                       574026 non-null  float64
 2   Alternative Dispute Resolution      574026 non-null  object 
 3   Assembly Date                       593471 non-null  object 
 4   Attorney/Representative             574026 non-null  object 
 5   Average Weekly Wage                 545375 non-null  float64
 6   Birth Year                          544948 non-null  float64
 7   C-2 Date                            559466 non-null  object 
 8   C-3 Date                            187245 non-null  object 
 9   Carrier Name                        574026 non-null  object 
 10  Carrier Type                        574026 non-null  object 
 11  Claim Identifier          

In [7]:
train_data.describe()

,Age at Injury,Average Weekly Wage,Birth Year,Claim Identifier,IME-4 Count,Industry Code,OIICS Nature of Injury Description,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Agreement Reached,Number of Dependents
count,574026.000000,5.453750e+05,544948.000000,5.934710e+05,132803.000000,564068.000000,0.0,558386.000000,558369.000000,556944.000000,574026.000000,574026.000000
mean,42.114270,4.910883e+02,1886.767604,2.366760e+07,3.207337,58.645305,NaN,54.381143,41.013839,39.738146,0.046665,3.006559
std,14.256432,6.092918e+03,414.644423,1.079271e+08,2.832303,19.644175,NaN,25.874281,22.207521,22.365940,0.210921,2.000801
min,0.000000,0.000000e+00,0.000000,5.393066e+06,1.000000,11.000000,NaN,1.000000,1.000000,-9.000000,0.000000,0.000000
25%,31.000000,0.000000e+00,1965.000000,5.593414e+06,1.000000,45.000000,NaN,31.000000,16.000000,33.000000,0.000000,1.000000
50%,42.000000,0.000000e+00,1977.000000,5.791212e+06,2.000000,61.000000,NaN,56.000000,49.000000,38.000000,0.000000,3.000000
75%,54.000000,8.410000e+02,1989.000000,5.991000e+06,4.000000,71.000000,NaN,75.000000,52.000000,53.000000,0.000000,5.000000
max,117.000000,2.828079e+06,2018.000000,9.998917e+08,73.000000,92.000000,NaN,99.000000,91.000000,99.000000,1.000000,6.000000


In [17]:
columns_name = train_data.columns
columns_name

Index(['Accident Date', 'Age at Injury', 'Alternative Dispute Resolution',
       'Assembly Date', 'Attorney/Representative', 'Average Weekly Wage',
       'Birth Year', 'C-2 Date', 'C-3 Date', 'Carrier Name', 'Carrier Type',
       'Claim Identifier', 'Claim Injury Type', 'County of Injury',
       'COVID-19 Indicator', 'District Name', 'First Hearing Date', 'Gender',
       'IME-4 Count', 'Industry Code', 'Industry Code Description',
       'Medical Fee Region', 'OIICS Nature of Injury Description',
       'WCIO Cause of Injury Code', 'WCIO Cause of Injury Description',
       'WCIO Nature of Injury Code', 'WCIO Nature of Injury Description',
       'WCIO Part Of Body Code', 'WCIO Part Of Body Description', 'Zip Code',
       'Agreement Reached', 'WCB Decision', 'Number of Dependents'],
      dtype='object')

In [19]:
for i in columns_name:
    print(i)
    print(train_data[i].unique())
    print(train_data[i].value_counts())
    print('\n')

Accident Date
['2019-12-30' '2019-08-30' '2019-12-06' ... '2000-05-24' '2007-05-01'
 '1980-05-17']
Accident Date
2020-03-01    1245
2020-12-18    1001
2022-02-07     977
2022-01-05     883
2021-02-18     851
              ... 
2017-05-17       1
2015-05-28       1
2002-08-18       1
2017-09-01       1
1980-05-17       1
Name: count, Length: 5539, dtype: int64


Age at Injury
[ 31.  46.  40.  nan  61.  67.  48.  33.  55.  20.  21.  51.  62.  35.
  54.  32.  34.  38.  30.  36.  39.  44.  56.  29.  60.  49.  50.  18.
  58.  24.  45.  53.  52.  47.  25.  23.  22.  64.  70.  41.  69.  57.
  27.  42.  77.  72.  19.  28.  63.  65.  59.  43.  26.  74.  66.  76.
  68.  37.   0.  16.  71.  82.  75.  17.  78.  73.  89.  85.  84.  81.
  87.  80.  11.  88.  79.  15.  83.  86.  90.   1.  10. 102.  94. 117.
  14. 113.  95.  92. 104.   5.   8. 115. 110.  91.  96.  99.  12.  13.
  93. 101.   7. 111.   9.  97. 112. 109. 100.  98. 114.]
Age at Injury
31.0     14041
30.0     14022
32.0     13994
29.0     

# Clean and pre-process the dataset

In [ ]:
""" X = titanic.drop('Survived', axis = 1)
y = titanic['Survived'] """

In [9]:
""" #filling with mean
for column in X_train.columns:
    if pd.api.types.is_numeric_dtype(X_train[column]):
        
        #store mean of training data in a variable - in a real application, you may need to store these values for future usages on e.g. test data 
        mean_to_fill = X_train[column].mean()
        
        #fill on X_train
        X_train[column].fillna(mean_to_fill, inplace=True) #X_train[column].fillna('Unknown')
        #Fill on X_val
        X_val[column].fillna(mean_to_fill, inplace=True) """

" #filling with mean\nfor column in X_train.columns:\n    if pd.api.types.is_numeric_dtype(X_train[column]):\n        \n        #store mean of training data in a variable - in a real application, you may need to store these values for future usages on e.g. test data \n        mean_to_fill = X_train[column].mean()\n        \n        #fill on X_train\n        X_train[column].fillna(mean_to_fill, inplace=True) #X_train[column].fillna('Unknown')\n        #Fill on X_val\n        X_val[column].fillna(mean_to_fill, inplace=True) "

In [10]:
""" X = train_data.drop(columns='Claim Injury Type', axis=1)
y = train_data['Claim Injury Type']

X_train, X_validation,y_train, y_validation = train_test_split(X,y,
                                                               train_size = 0.7,
                                                               random_state=0,
                                                               shuffle = True, 
                                                               stratify = y) """

" X = train_data.drop(columns='Claim Injury Type', axis=1)\ny = train_data['Claim Injury Type']\n\nX_train, X_validation,y_train, y_validation = train_test_split(X,y,\n                                                               train_size = 0.7,\n                                                               random_state=0,\n                                                               shuffle = True, \n                                                               stratify = y) "

In [ ]:
""" X_train_num = X_train.select_dtypes(include=np.number).set_index(X_train.index)
X_train_num.drop('Embarked_Flag', axis = 1, inplace = True)

X_train_cat = X_train.select_dtypes(exclude=np.number).set_index(X_train.index)
X_train_cat['Embarked_Flag'] = X_train['Embarked_Flag']

# Repeat for Validation
#CODE HERE
X_val_num = X_val.select_dtypes(include=np.number).set_index(X_val.index)
X_val_num.drop('Embarked_Flag', axis = 1, inplace = True)

X_val_cat = X_val.select_dtypes(exclude=np.number).set_index(X_val.index)
X_val_cat['Embarked_Flag'] = X_val['Embarked_Flag'] """

In [ ]:
""" #imported here for demonstrative purposes but it is a good practice to import everything at the start.
from sklearn.preprocessing import MinMaxScaler

#call function
scaler = MinMaxScaler()

#fit to training data
scaler.fit(X_train_num)

#transform the data
X_train_num_scaled = scaler.transform(X_train_num) # this will return an array

#show results
X_train_num_scaled """

In [ ]:
""" # Convert the array to a pandas dataframe
X_train_num_scaled = pd.DataFrame(X_train_num_scaled, columns = X_train_num.columns).set_index(X_train.index)
X_train_num_scaled """

In [ ]:
""" X_val_num_scaled = scaler.transform(X_val_num)
X_val_num_scaled = pd.DataFrame(X_val_num_scaled, columns = X_val_num.columns).set_index(X_val.index)
X_val_num_scaled.head(3) """

# Feature selection

# Build a simple model and assess the performance

In [11]:
""" dt = DecisionTreeClassifier().fit(X_train, y_train)
predictions_train = dt.predict(X_train)
predictions_val = dt.predict(X_validation) """

' dt = DecisionTreeClassifier().fit(X_train, y_train)\npredictions_train = dt.predict(X_train)\npredictions_val = dt.predict(X_validation) '

In [12]:
""" print(dt.score(X_train, y_train))
print(dt.score(X_validation, y_validation)) """

' print(dt.score(X_train, y_train))\nprint(dt.score(X_validation, y_validation)) '

In [13]:
""" print(confusion_matrix(y_train, predictions_train))
print(confusion_matrix(y_validation, predictions_val)) """

' print(confusion_matrix(y_train, predictions_train))\nprint(confusion_matrix(y_validation, predictions_val)) '

In [14]:
""" drugs_2classify['DrugPlant'] = dt.predict(drugs_2classify)
drugs_2classify

#export test data predictions
drugs_2classify['DrugPlant'].to_csv('Exercise1_predictions.csv') """

" drugs_2classify['DrugPlant'] = dt.predict(drugs_2classify)\ndrugs_2classify\n\n#export test data predictions\ndrugs_2classify['DrugPlant'].to_csv('Exercise1_predictions.csv') "